In [1]:
import requests
import html
from bs4 import BeautifulSoup
from IPython.display import display

In [34]:
url = 'https://myanimelist.net/api/anime/search.xml?q='
query = 'bleach'
user='*******'
password='******'
response = requests.get(url+query, auth=requests.auth.HTTPBasicAuth(user, password))
soup = BeautifulSoup(response.text, 'html.parser')

print(html.unescape(soup.synopsis.get_text()))

Ichigo Kurosaki is an ordinary high schooler—until his family is attacked by a Hollow, a corrupt spirit that seeks to devour human souls. It is then that he meets a Soul Reaper named Rukia Kuchiki, who gets injured while protecting Ichigo's family from the assailant. To save his family, Ichigo accepts Rukia's offer of taking her powers and becomes a Soul Reaper as a result.<br />
<br />
However, as Rukia is unable to regain her powers, Ichigo is given the daunting task of hunting down the Hollows that plague their town. However, he is not alone in his fight, as he is later joined by his friends—classmates Orihime Inoue, Yasutora Sado, and Uryuu Ishida—who each have their own unique abilities. As Ichigo and his comrades get used to their new duties and support each other on and off the battlefield, the young Soul Reaper soon learns that the Hollows are not the only real threat to the human world.<br />
<br />
[Written by MAL Rewrite]


In [125]:
import requests
import html
from bs4 import BeautifulSoup
from IPython.display import display
import re
import pandas as pd

In [136]:
def cleanse_data(tag):
    text=tag.get_text()
    pos,title,tv,date,members,rating,na,na = re.findall(r"\s*(.+)",text)
    match = re.search(r'(.+)Watch ((Episode)|(Promotional)) Video',title)
    if match:
        title = match.group(1)
    return (int(pos), title, date.split('-')[0][:-1], date.split('-')[1][1:], float(rating))

top_animes_url = 'https://myanimelist.net/topanime.php'
top_animes_soup = BeautifulSoup(requests.get(top_animes_url).text, 'html.parser')
next_limit = top_animes_soup.find("a", class_='next')
if next_limit:
    next_limit = next_limit.get('href')

tags = top_animes_soup.find_all("tr", class_='ranking-list')

entries = {'pos':[], 'title':[], 'start_date':[], 'end_date':[], 'rating':[]}
for block in map(cleanse_data,tags):
    entries['pos'].append(block[0])
    entries['title'].append(block[1])
    entries['start_date'].append(block[2])
    entries['end_date'].append(block[3])
    entries['rating'].append(block[4])
    
while next_limit:
# for i in range(3):
    top_animes_soup = BeautifulSoup(requests.get(top_animes_url + next_limit ).text, 'html.parser')
    tags = top_animes_soup.find_all("tr", class_='ranking-list')
    for block in map(cleanse_data,tags):
        entries['pos'].append(block[0])
        entries['title'].append(block[1])
        entries['start_date'].append(block[2])
        entries['end_date'].append(block[3])
        entries['rating'].append(block[4])
    next_limit = top_animes_soup.find("a", class_='next')
    if next_limit:
        next_limit=next_limit.get('href')

anime_db = pd.DataFrame.from_dict(entries)

In [137]:
anime_db.shape

(2250, 5)

In [138]:
anime_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250 entries, 0 to 2249
Data columns (total 5 columns):
end_date      2250 non-null object
pos           2250 non-null int64
rating        2250 non-null float64
start_date    2250 non-null object
title         2250 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 88.0+ KB


In [139]:
anime_db.to_csv('anime_ranking')